In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import NullFormatter
import calculus as cal

np.set_printoptions(precision=4, suppress=True)

In [ ]:
# define the broken/good cobras
#brokens = [1, 2, 39, 43, 54]
brokens = [1,39,43,54]
visibles= [e for e in range(1,58) if e not in brokens]
badIdx = np.array(brokens) - 1
goodIdx = np.array(visibles) - 1
mod = 'SP01'

# two groups for two cameras
cam_split = 28
group1 = goodIdx[goodIdx <= cam_split]
group2 = goodIdx[goodIdx > cam_split]

In [ ]:
path = '/data/MCS/20201203_026/data/'
#path = '/home/pfs/Documents/cwen/data/20200504010/data/'
centers = np.load(path + 'thetaCenter.npy')
radius = np.load(path + 'thetaRadius.npy')
fw = np.load(path + 'thetaFW.npy')
rv = np.load(path + 'thetaRV.npy')
af = np.load(path + 'thetaAngFW.npy')
ar = np.load(path + 'thetaAngRV.npy')
sf = np.load(path + 'thetaSpeedFW.npy')
sr = np.load(path + 'thetaSpeedRV.npy')
badR = np.load(path + 'badRange.npy')
mf = np.load(path + 'thetaMMFW.npy')
mr = np.load(path + 'thetaMMRV.npy')
badM = np.load(path + 'badMotorMap.npy')
badR = np.load(path + 'badRange.npy')
steps = int(np.load(path + 'steps.npy'))
ontime = np.load(path + 'ontime.npy')

In [ ]:
plt.figure(figsize=(20,8))
plt.clf()

plt.subplot(211)
ax = plt.gca()
ax.axis('equal')

ax.plot(centers[group1].real, centers[group1].imag, 'ro')
for idx in group1:
    c = plt.Circle((centers[idx].real, centers[idx].imag), radius[idx], color='g', fill=False)
    ax.add_artist(c)
ax.set_title(f'1st camera')

plt.subplot(212)
ax = plt.gca()
ax.axis('equal')

ax.plot(centers[group2].real, centers[group2].imag, 'ro')
for idx in group2:
    c = plt.Circle((centers[idx].real, centers[idx].imag), radius[idx], color='g', fill=False)
    ax.add_artist(c)
ax.set_title(f'2nd camera')

plt.show()

In [ ]:
plt.figure(figsize=(20,8))
plt.clf()

plt.subplot(211)
ax = plt.gca()
ax.axis('equal')
ax.set_title(f'1st camera')

for n in range(fw.shape[1]):
    for k in group1:
        if k % 3 == 0:
            c = 'r'
            d = 'c'
        elif k % 3 == 1:
            c = 'g'
            d = 'm'
        else:
            c = 'b'
            d = 'y'
        ax.plot(fw[k][n,0].real, fw[k][n,0].imag, c + 'o')
        ax.plot(rv[k][n,0].real, rv[k][n,0].imag, d + 's')
        ax.plot(fw[k][n,1:].real, fw[k][n,1:].imag, c + '.')
        ax.plot(rv[k][n,1:].real, rv[k][n,1:].imag, d + '.')

plt.subplot(212)
ax = plt.gca()
ax.axis('equal')
ax.set_title(f'2nd camera')

for n in range(fw.shape[1]):
    for k in group2:
        if k % 3 == 0:
            c = 'r'
            d = 'c'
        elif k % 3 == 1:
            c = 'g'
            d = 'm'
        else:
            c = 'b'
            d = 'y'
        ax.plot(fw[k][n,0].real, fw[k][n,0].imag, c + 'o')
        ax.plot(rv[k][n,0].real, rv[k][n,0].imag, d + 's')
        ax.plot(fw[k][n,1:].real, fw[k][n,1:].imag, c + '.')
        ax.plot(rv[k][n,1:].real, rv[k][n,1:].imag, d + '.')

plt.show()

In [ ]:
plt.figure(figsize=(16,16))
ax = plt.gca()
ax.axis('equal')
k = 29
ax.set_title(f'Cobra #{k+1}')

c = plt.Circle((centers[k].real, centers[k].imag), radius[k], fill=False)
ax.add_artist(c)

for n in range(fw.shape[1]):
    c = 'r'
    d = 'b'
    ax.plot(fw[k][n,1:].real, fw[k][n,1:].imag, c + '.')
    ax.plot(rv[k][n,1:].real, rv[k][n,1:].imag, d + '.')
    ax.plot(fw[k][n,0].real, fw[k][n,0].imag, c + 'x')
    ax.plot(rv[k][n,0].real, rv[k][n,0].imag, d + '+')

In [ ]:
with PdfPages(f'pdf/{mod}_theta_measure2.pdf') as pdf:
    for k in range(57):
        fig = plt.figure(figsize=(16,16))
        ax = plt.gca()
        ax.axis('equal')
        ax.set_title(f'Cobra #{k+1}')

        c = plt.Circle((centers[k].real, centers[k].imag), radius[k], fill=False)
        ax.add_artist(c)

        for n in range(fw.shape[1]):
            c = 'r'
            d = 'b'
            ax.plot(fw[k][n,0].real, fw[k][n,0].imag, c + 'x')
            ax.plot(rv[k][n,0].real, rv[k][n,0].imag, d + '+')
            ax.plot(fw[k][n,1:].real, fw[k][n,1:].imag, c + '.')
            ax.plot(rv[k][n,1:].real, rv[k][n,1:].imag, d + '.')

        pdf.savefig(fig)
        plt.close()

In [ ]:
with PdfPages(f'pdf/{mod}_theta_{steps}.pdf') as pdf:
    for c in range(57):
        fig = plt.figure(figsize=(16,6))
        ax = plt.gca()
        ax.set_title(f'Cobra {mod} #{c+1} THETA @ {steps}steps')
        for n in range(af.shape[1]):
            ax.plot(np.rad2deg(af[c,n,:-1]), np.rad2deg(af[c,n,1:]-af[c,n,:-1])/steps, '.')
            ax.plot(np.rad2deg(ar[c,n,1:]), np.rad2deg(ar[c,n,1:]-ar[c,n,:-1])/steps, '.')
            ax.plot([0,380], [0,0], 'r')
        bottom, top = plt.ylim()
        plt.ylim([max(-0.2,bottom), min(0.2,top)])
        pdf.savefig(fig)
        plt.close()

In [ ]:
for c in goodIdx:
    fig = plt.figure(figsize=(16,6))
    ax = plt.gca()
    ax.set_title(f'Cobra {mod} #{c+1} THETA @ {steps}steps')
    for n in range(af.shape[1]):
        ax.plot(np.rad2deg(af[c,n,:-1]), np.rad2deg(af[c,n,1:]-af[c,n,:-1])/steps, '.')
        ax.plot(np.rad2deg(ar[c,n,1:]), np.rad2deg(ar[c,n,1:]-ar[c,n,:-1])/steps, '.')
        ax.plot([0,380], [0,0], 'r')
#    bottom, top = plt.ylim()
#    plt.ylim([max(-0.2,bottom), min(0.2,top)])
    plt.show()
    plt.close()

In [ ]:
for c in range(57):
#    plt.figure(3)
#    plt.clf()
    fig = plt.figure(figsize=(16,6))
    ax = plt.gca()
    ax.set_title(f'#{c}')
    for n in range(af.shape[1]):
        ax.plot(af[c, n], '.')
        ax.plot(ar[c, n], '.')
    plt.show()
    plt.close()

In [ ]:
plt.figure(4)
plt.clf()
ax = plt.gca()

ax.plot(np.rad2deg(sf), '.')
ax.plot(np.rad2deg(sr), 'x')
#ax.plot(sf2, '.')
#ax.plot(sr2, 'x')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(16,12))

#ax1 = plt.gca()
ax1.set_title('Phi 60, Run 1')
ax1.plot(np.rad2deg(sf), '.')
ax1.plot(np.rad2deg(sr), 'x')

#plt.figure(figsize=(12,8))
#ax = plt.gca()
ax2.set_title('Phi 60, Run 2')
ax2.plot(np.rad2deg(sf2), '.')
ax2.plot(np.rad2deg(sr2), 'x')

#plt.figure(figsize=(12,8))
#ax = plt.gca()
ax3.set_title('Phi 60, Run 1 - Run 2')
ax3.plot(np.rad2deg(sf-sf2), '.')
ax3.plot(np.rad2deg(sr-sr2), 'x')

with PdfPages(f'pdf/{mod}_theta_60.pdf') as pdf:
    pdf.savefig()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(16,12))

#ax1 = plt.gca()
ax1.set_title('Phi 180, Run 1')
ax1.plot(np.rad2deg(sf3), '.')
ax1.plot(np.rad2deg(sr3), 'x')

#plt.figure(figsize=(12,8))
#ax = plt.gca()
ax2.set_title('Phi 180, Run 2')
ax2.plot(np.rad2deg(sf4), '.')
ax2.plot(np.rad2deg(sr4), 'x')

#plt.figure(figsize=(12,8))
#ax = plt.gca()
ax3.set_title('Phi 180, Run 1 - Run 2')
ax3.plot(np.rad2deg(sf3-sf4), '.')
ax3.plot(np.rad2deg(sr3-sr4), 'x')

with PdfPages(f'pdf/{mod}_theta_180.pdf') as pdf:
    pdf.savefig()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(16,12))

sfa = (sf+sf2)/2
sra = (sr+sr2)/2

sfb = (sf3+sf4)/2
srb = (sr3+sr4)/2

#ax1 = plt.gca()
ax1.set_title('Phi 60, average')
ax1.plot(np.rad2deg(sfa), '.')
ax1.plot(np.rad2deg(sra), 'x')

#plt.figure(figsize=(12,8))
#ax = plt.gca()
ax2.set_title('Phi 180, average')
ax2.plot(np.rad2deg(sfb), '.')
ax2.plot(np.rad2deg(srb), 'x')

#plt.figure(figsize=(12,8))
#ax = plt.gca()
ax3.set_title('Phi 60 - Phi 180')
ax3.plot(np.rad2deg(sfa-sfb), '.')
ax3.plot(np.rad2deg(sra-srb), 'x')

with PdfPages(f'pdf/{mod}_theta_60-180.pdf') as pdf:
    pdf.savefig()

In [ ]:
paths = ['/data/MCS/20200506_016/data/','/data/MCS/20200506_017/data/','/data/MCS/20200506_018/data/','/data/MCS/20200506_019/data/','/data/MCS/20200506_020/data/','/data/MCS/20200506_021/data/','/data/MCS/20200506_022/data/','/data/MCS/20200508_001/data/']
n = len(paths)
sf = np.zeros((57,n))
sr = np.zeros((57,n))
steps = np.zeros(n, 'int')
ontime = np.zeros((57,2,n))

for m,path in enumerate(paths):
    sf[:,m] = np.load(path + 'thetaSpeedFW.npy')
    sr[:,m] = np.load(path + 'thetaSpeedRV.npy')
    steps[m] = int(np.load(path + 'steps.npy'))
    ontime[:,:,m] = np.load(path + 'ontime.npy').T

In [ ]:
nsf = np.zeros((57,n))
nsr = np.zeros((57,n))
med = np.zeros(n)

for c in range(57):
    nsf[c] = sf[c] / sf[c,0]
    nsr[c] = sr[c] / sr[c,0]

for m in range(n):
    med[m] = np.median([nsf[:,m], nsr[:,m]])

In [ ]:
with PdfPages(f'pdf/{mod}_theta_speed.pdf') as pdf:
    fig = plt.figure(figsize=(16,6))
    plt.ylim([0,2])
    plt.xticks(np.arange(n), steps)
    for c in range(57):
    #c = 10
        plt.plot(sf[c]/sf[c,0], '.')
        plt.plot(sr[c]/sr[c,0], '.')
    pdf.savefig()
    plt.show()
    plt.close()

    fig = plt.figure(figsize=(16,6))
    plt.xticks(np.arange(n), steps)
    plt.plot(med, 'x')
    pdf.savefig()
    plt.show()
    plt.close()

In [ ]:
bad, badR

In [ ]:
c = 17

plt.figure(5)
plt.clf()

#plt.subplot(211)
ax = plt.gca()
ax.set_title(f'#{c} FW')
ax.plot(np.rad2deg(mf[c]), 'r')


#plt.subplot(212)
ax = plt.gca()
ax.set_title(f'#{c} RV')
ax.plot(np.rad2deg(mr[c]), 'b')

In [ ]:
c = 30

plt.figure(5)
plt.clf()

#plt.subplot(211)
ax = plt.gca()
ax.set_title(f'#{c} FW')
ax.plot(mf2[c], 'r')


#plt.subplot(212)
ax = plt.gca()
ax.set_title(f'#{c} RV')
ax.plot(mr2[c], 'b')

In [ ]:
from ics.cobraCharmer import pfiDesign
import pathlib

In [ ]:
#xml = pathlib.Path('/home/pfs/Documents/cwen/ics_cobraCharmer/xml/SC04_03.xml')
xml = pathlib.Path('/data/MCS/20200110_003/output/SC34_theta_400.xml')
model = pfiDesign.PFIDesign(xml)

In [ ]:
model.motorOntimeSlowFwd1, model.motorOntimeSlowRev1

In [ ]:
(model.tht1-model.tht0) % (np.pi*2), np.rad2deg((model.tht1-model.tht0) % (np.pi*2)), np.rad2deg(6.2)

In [ ]:
np.rad2deg(model.tht0[45:48]), np.rad2deg(model.tht1[45:48])

In [ ]:
tf = model.motorOntimeSlowFwd1.copy()
tr = model.motorOntimeSlowRev1.copy()

In [ ]:
np.where(tf>0.07), np.where(tr>0.07)

In [ ]:
tf, tr

In [ ]:
spd_adj = np.deg2rad(0.075)
tf /= 1+(sf/spd_adj-1)/3
tr /= 1+(sr/spd_adj-1)/3

In [ ]:
tf, tr

In [ ]:
tf[31] = 0.06
tr[36] = 0.05

In [ ]:
np.where(np.rad2deg(sf) < 0.05)

In [ ]:
tr[41] = 0.06
tr[56] = 0.05

In [ ]:
tr[np.rad2deg(sr) < 0.05] *= 1.2

In [ ]:
tf[10] = 0.05

In [ ]:
tf, tr

In [ ]:
tr[41], tr[56]

In [ ]:
model.motorOntimeSlowFwd1[17], model.motorOntimeSlowRev1[17]

In [ ]:
model.updateOntimes(thetaFwd=tf, thetaRev=tr, fast=False)

In [ ]:
model.createCalibrationFile('/home/pfs/Documents/cwen/ics_cobraCharmer/xml/SC34_02.xml')

In [ ]:
tf[13], tr[13]

In [ ]:
tf[5] = 0.04
tr[5] = 0.04
tf[30] = 0.035

In [ ]:
fw[19], rv[19]

In [ ]:
tr[13] = 0.045

In [ ]:
ar[1,0]

In [ ]:
np.argmin(abs(ar[1,0]%(np.pi*2) - np.pi))

In [ ]:
fw[30], rv[30]

In [ ]:
fw[43], rv[43], centers[43]

In [ ]:
data = np.concatenate((fw[33].flatten(), rv[33].flatten()))

In [ ]:
np.max(np.real(data)), np.min(np.real(data)), np.max(np.imag(data)), np.min(np.imag(data))

In [ ]:
centers[33]

In [ ]:
data = rv[33, 1].flatten()
plt.plot(np.real(data), np.imag(data), '.')

In [ ]:
model.centers, centers, model.centers - centers

In [ ]:
path = '/data/MCS/20191230_003/data/'
data = np.load(path + 'thetaData.npy')

In [ ]:
angles = np.linspace(15,345,34)
print(np.where(abs(np.rad2deg(data[goodIdx,:,7,0])-angles)>0.2))
print(np.where(abs(np.rad2deg(data[goodIdx,:,7,0])-angles)>0.1))
print(np.where(abs(np.rad2deg(data[goodIdx,:,7,0])-angles)>0.1)[0].shape[0])
print(np.amax(abs(np.rad2deg(data[goodIdx,:,:,0])-angles[:,np.newaxis]), axis=(0,1)))
np.average(abs(np.rad2deg(data[goodIdx,:,:,0])-angles[:,np.newaxis]), axis=(0,1))

In [ ]:
c = 12

plt.figure(4)
plt.clf()
for n in range(data.shape[1]):
    plt.plot(np.rad2deg(data[c,n,:,0]))

In [ ]:
#bad = [37]
#useable= [e for e in range(1,58) if e not in bad]
#idx = np.array(useable) - 1
idx = goodIdx

for run in range(8):
#    plt.figure()
    d = (np.rad2deg(data[idx,:,:,0])-angles[:,np.newaxis])[:,:,run]

    plt.subplot(121)
#    plt.xscale('log')
    plt.yscale('log')
    plt.title(f'Diff angles #{run+1}')
#    plt.grid(True)
    plt.hist(d.flatten(), bins='auto')

    plt.subplot(122)
    plt.xscale('log')
    plt.yscale('log')
    plt.title(f'Absolute diffs #{run+1}')
    plt.grid(True)
    plt.hist(abs(d).flatten(), bins='auto')

    plt.gca().yaxis.set_minor_formatter(NullFormatter())
    plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.45, wspace=0.35)
    with PdfPages(f'pdf/SC31_theta_80_{run+1}.pdf') as pdf:
        pdf.savefig()
    plt.show()

In [ ]:
datadir = '/data/MCS/20201216_003'
angles = np.load(f'{datadir}/data/angles.npy')
ontimes = np.load(f'{datadir}/data/ontimes.npy')
speeds = np.load(f'{datadir}/data/speeds.npy')
goodIdx = np.load(f'{datadir}/data/cobras.npy')
parameters = np.load(f'{datadir}/data/parameters.npy')
mm = np.load(f'{datadir}/data/thetaOntimeMap.npy')

In [ ]:
with PdfPages(f'pdf/{mod}_theta_scan_fast5.pdf') as pdf:
    for idx in range(len(ontimes)):
        fig = plt.figure(figsize=(20,30))
        ax1 = plt.subplot(411)
        ax1.set_title(f'Cobra #{goodIdx[idx]+1}: On-time (theta, Forward)')
        ax2 = plt.subplot(412)
        ax2.set_title(f'Cobra #{goodIdx[idx]+1}: Step size (theta, Forward)')
        ax3 = plt.subplot(413)
        ax3.set_title(f'Cobra #{goodIdx[idx]+1}: On-time (theta, Backward)')
        ax4 = plt.subplot(414)
        ax4.set_title(f'Cobra #{goodIdx[idx]+1}: Step size (theta, Backward)')

        for repeat in range(ontimes.shape[1]):
            data = ontimes[idx,repeat,0]*1000
            nz = np.where(data == 0)[0]
            if len(nz) > 0:
                upper = np.where(data == 0)[0][0] - 1
            else:
                upper = len(data)
            xdata = np.rad2deg(angles)[idx,repeat,0,:upper]
            data1 = np.rad2deg(speeds[idx,repeat,0])
            ax1.plot(xdata, data[:upper], '.', label=f'{repeat+1}')
            ax2.plot(xdata, data1[:upper], '.', label=f'{repeat+1}')

            data = ontimes[idx,repeat,1]*1000
            nz = np.where(data == 0)[0]
            if len(nz) > 0:
                upper = np.where(data == 0)[0][0]
            else:
                upper = len(data)
            xdata = np.rad2deg(angles)[idx,repeat,1,:upper]
            data1 = -np.rad2deg(speeds[idx,repeat,1])
            ax3.plot(xdata, data[:upper], '.', label=f'{repeat+1}')
            ax4.plot(xdata, data1[:upper], '.', label=f'{repeat+1}')

        ax1.legend()
        ax2.legend()
        ax3.legend()
        ax4.legend()
        pdf.savefig(fig)
        plt.close()

In [ ]:
with PdfPages(f'pdf/{mod}_theta_mm_fast2.pdf') as pdf:
    for c in goodIdx:
        fig = plt.figure(figsize=(15,20))
        xdata = np.rad2deg(mm['angle'][c])
        ax1 = plt.subplot(411)
        ax1.set_title(f'Cobra #{c+1}: On-time (theta, Forward)')
        ax1.plot(xdata[0], mm['ontime'][c,0], 'x')
        ax2 = plt.subplot(412)
        ax2.set_title(f'Cobra #{c+1}: Speed (theta, Forward)')
        ax2.plot(xdata[0], np.rad2deg(mm['speed'][c,0]), 'x')
        ax3 = plt.subplot(413)
        ax3.set_title(f'Cobra #{c+1}: On-time (theta, Reverse)')
        ax3.plot(xdata[1], mm['ontime'][c,1], 'x')
        ax4 = plt.subplot(414)
        ax4.set_title(f'Cobra #{c+1}: Speed (theta, Reverse)')
        ax4.plot(xdata[1], np.rad2deg(-mm['speed'][c,1]), 'x')

        pdf.savefig(fig)
        plt.close()

In [ ]:
datadir = '/home/cwen/mhs/devel/ics_cobraCharmer/procedures/moduleTest'
mm2 = np.load(f'{datadir}/SP01_mmThetaFast5.npy')
mm3 = np.load(f'{datadir}/SP01_mmThetaSlow.npy')

with PdfPages(f'pdf/{mod}_theta_mm_fast_compare2.pdf') as pdf:
    for c in goodIdx:
        fig = plt.figure(figsize=(15,20))
        ax1 = plt.subplot(411)
        ax1.set_title(f'Cobra #{c+1}: On-time (theta, Forward)')
        ax2 = plt.subplot(412)
        ax2.set_title(f'Cobra #{c+1}: Speed (theta, Forward)')
        ax3 = plt.subplot(413)
        ax3.set_title(f'Cobra #{c+1}: On-time (theta, Reverse)')
        ax4 = plt.subplot(414)
        ax4.set_title(f'Cobra #{c+1}: Speed (theta, Reverse)')
        for color,mmx in [('m',mm2), ('b',mm3)]:
            xdata = np.rad2deg(mmx['angle'][c])
            ax1.plot(xdata[0], mmx['ontime'][c,0], f'x{color}')
            ax2.plot(xdata[0], np.rad2deg(mmx['speed'][c,0]), f'x{color}')
            ax3.plot(xdata[1], mmx['ontime'][c,1], f'x{color}')
            ax4.plot(xdata[1], np.rad2deg(-mmx['speed'][c,1]), f'x{color}')

        pdf.savefig(fig)
        plt.close()

In [ ]:
def smooth(x, window_len=21, window='hamming'):
    s = np.r_[x[window_len-1:0:-1], x, x[-2:-window_len-1:-1]]
    w = eval('np.' + window + '(window_len)')
    y = np.convolve(w/w.sum(), s, mode='valid')
    return y[(window_len//2):-(window_len//2)]

In [ ]:
idx = 50

plt.figure(figsize=(15,30))
ax1 = plt.subplot(411)
ax1.set_title(f'Cobra #{goodIdx[idx]+1}: On-time (theta, Forward)')
ax2 = plt.subplot(412)
ax2.set_title(f'Cobra #{goodIdx[idx]+1}: Step size (theta, Forward)')
ax3 = plt.subplot(413)
ax3.set_title(f'Cobra #{goodIdx[idx]+1}: On-time (theta, Backward)')
ax4 = plt.subplot(414)
ax4.set_title(f'Cobra #{goodIdx[idx]+1}: Step size (theta, Backward)')

#for repeat in range(ontimes.shape[1]):
for repeat in range(1):
    data = ontimes[idx,repeat,0]*1000
    nz = np.where(data == 0)[0]
    if len(nz) > 0:
        upper = nz[0] - 1
    else:
        upper = len(data)
    xdata = np.rad2deg(angles)[idx,repeat,0,:upper]
    data1 = np.rad2deg(speeds[idx,repeat,0])
    ax1.plot(xdata, data[:upper], '.', label=f'{repeat+1}')
    ax1.plot(xdata, smooth(data[:upper]), label=f'S{repeat+1}')
    ax2.plot(xdata, data1[:upper], '.', label=f'{repeat+1}')
    ax2.plot(xdata, smooth(data1[:upper]), label=f'S{repeat+1}')

    data = ontimes[idx,repeat,1]*1000
    nz = np.where(data == 0)[0]
    if len(nz) > 0:
        upper = nz[0]
    else:
        upper = len(data)
    xdata = np.rad2deg(angles)[idx,repeat,1,:upper]
    data1 = -np.rad2deg(speeds[idx,repeat,1])
    ax3.plot(xdata, data[:upper], '.', label=f'{repeat+1}')
    ax3.plot(xdata, smooth(data[:upper]), label=f'S{repeat+1}')
    ax4.plot(xdata, data1[:upper], '.', label=f'{repeat+1}')
    ax4.plot(xdata, smooth(data1[:upper]), label=f'S{repeat+1}')

ax1.legend()
ax2.legend()
ax3.legend()
ax4.legend()
plt.show()

In [ ]:
mm[1,0]

In [ ]:
mmDtype = np.dtype(dict(names=['angle', 'ontime', 'speed'], formats=['f4', 'f4', 'f4']))

In [ ]:
mm = np.full((angles.shape[0],2,angles.shape[3]-1), np.nan, dtype=mmDtype)

for i in range(angles.shape[0]):
    for j in range(2):
        nz = np.where(ontimes[i,0,j] == 0)[0]
        if len(nz) > 0:
            upper = nz[0] - 1
        else:
            upper = ontimes.shape[3] - 1
        mm[i,j,:upper]['angle'] = angles[i,0,j,:upper]
        mm[i,j,:upper]['ontime'] = smooth(ontimes[i,0,j,:upper])
        mm[i,j,:upper]['speed'] = smooth(speeds[i,0,j,:upper])

In [ ]:
mmOut = np.full((57,2,angles.shape[3]-1), 0, dtype=mmDtype)
mmOut[goodIdx] = mm
np.save('SP01_mmThetaFast4', mmOut)

In [ ]:
angMax = np.nanmax(mm['angle'], axis=2)
angMin = np.nanmin(mm['angle'], axis=2)
angMargin = np.deg2rad(15)
ot = np.zeros((mm.shape[0],2))

for i in range(mm.shape[0]):
    for j in range(2):
        x = mm[i,j]['angle']
        mask = ~np.isnan(x)
        mask[mask] &= x[mask] > angMin[i,j] + angMargin
        mask[mask] &= x[mask] < angMax[i,j] - angMargin
        ot[i,j] = np.nanmax(mm[i,j,mask]['ontime']) + np.nanmin(mm[i,j,mask]['ontime'])*1.0

ot[ot > 0.12] = 0.12

In [ ]:
otOut = np.copy(ontime)
otOut[:,goodIdx] = ot.T
otOut

In [ ]:
moves = np.load('/data/MCS/20201207_051/data/moves.npy')
positions = np.load('/data/MCS/20201207_051/data/positions.npy')
targets = np.load('/data/MCS/20201207_051/data/targets.npy')

In [ ]:
moves.shape, positions.shape, targets.shape

In [ ]:
run = 1
c = 4
positions[run,c], targets[run,c], moves[run,c,:]

In [ ]:
segs = np.load('/data/MCS/20201207_033/data/segments_1607313086.npz')

In [ ]:
segs['idx'], segs['segs'][:,4]

In [ ]:
#centers = np.zeros((3,57), 'complex')
#radius = np.zeros((3,57), 'float')
centers = []
radius = []
fw = []
rv = []

for n in [2,4,7,9]:
    path = f'/data/MCS/20201204_{n:03}/data/'
    centers.append(np.load(path + 'thetaCenter.npy'))
    radius.append(np.load(path + 'thetaRadius.npy'))
    fw.append(np.load(path + 'thetaFW.npy'))
    rv.append(np.load(path + 'thetaRV.npy'))

In [ ]:
with PdfPages(f'pdf/{mod}_theta_centers2.pdf') as pdf:
    for k in goodIdx:
        fig = plt.figure(figsize=(16,16))
        ax = plt.gca()
        ax.axis('equal')
        ax.set_title(f'Cobra #{k+1}')

        for m in range(len(centers)):
            c = plt.Circle((centers[m][k].real, centers[m][k].imag), radius[m][k], fill=False)
            ax.add_artist(c)
            ax.plot(centers[m][k].real, centers[m][k].imag, 'o')

            for n in range(fw[m].shape[1]):
                c = 'r'
                d = 'b'
                ax.plot(fw[m][k][n,1:].real, fw[m][k][n,1:].imag, c + '.')
                ax.plot(rv[m][k][n,1:].real, rv[m][k][n,1:].imag, d + '.')
                ax.plot(fw[m][k][n,0].real, fw[m][k][n,0].imag, c + 'x')
                ax.plot(rv[m][k][n,0].real, rv[m][k][n,0].imag, d + '+')

        pdf.savefig(fig)
        plt.close()

In [ ]:
np.average(centers, axis=0)

In [ ]:
fw[27:32,0,15]

In [ ]:
from ics.cobraCharmer import pfiDesign
import pathlib

In [ ]:
xml = pathlib.Path('/home/cwen/mhs/devel/pfs_instdata/data/pfi/modules/SP01/SP01.xml')
model = pfiDesign.PFIDesign(xml)

In [ ]:
c = 0
model.thtOffsets[c], model.posThtSteps[c], model.F1Pm[c], model.negThtSteps[c], model.F1Nm[c]

In [ ]:
c = 0
thtSteps = model.posThtSteps[c]
np.interp([0.0628, 0.1885], model.thtOffsets[c], thtSteps)

In [ ]:
np.interp(0.03, model.thtOffsets[c], thtSteps)

In [ ]:
np.interp(125, thtSteps, model.thtOffsets[c])

In [ ]:
np.rad2deg(0.6)